0. 所有的题目结果中，给出SQL语句和执行结果。

1. 在新数据库中新建一张 user 表,插入几条数据,属性包含:唯一标识(id),姓名(name)性别(sex).年龄(age).联系方式(phone)，数据如下：

    ('John Doe', 'Male', 25, '123-456-7890')

    ('Jane Smith', 'Female', 31, '987-654-3210')
    
    ('Bob Johnson', 'Male', 22, '555-123-4567')

In [1]:
import sqlite3 as db
conn = db.connect('user_database.db')
cursor = conn.cursor()

cursor.execute('DELETE FROM user')#防止重复添加
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS user (
    id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, sex TEXT, age INTEGER, phone TEXT)
''')

cursor.executemany('''
INSERT INTO user (name, sex, age, phone) VALUES (?, ?, ?, ?)''', 
[   ('John Doe', 'Male', 25, '123-456-7890'),
    ('Jane Smith', 'Female', 31, '987-654-3210'),
    ('Bob Johnson', 'Male', 22, '555-123-4567')
])

conn.commit()

2. 写出 SQL语句,查询 user 表中所有年龄在 20-30 范围内的用户

In [2]:
import pandas as pd

query = "SELECT * FROM user WHERE age BETWEEN 20 AND 30"
result = pd.read_sql_query(query, conn)
print(result)

   id         name   sex  age         phone
0   1     John Doe  Male   25  123-456-7890
1   3  Bob Johnson  Male   22  555-123-4567


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/local/lib/python3.10/dist-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


3. 写出SQL语句，向user表中添加自己的个人信息，并添加几条和你姓名同姓的虚拟信息。

In [3]:
cursor.executemany('''
INSERT INTO user (name, sex, age, phone) VALUES (?, ?, ?, ?)''',
 [  ('Zyan Li', 'Male', 19, '123-456-7777'),
    ('Zyan Li', 'Male', 24, '987-654-0000'),
    ('Zyan Li', 'Male', 22, '555-123-6666')
])

conn.commit()

4. 写出 SQL语句,查询 user 表中年龄在 20-30 范围内,名字包含“你的姓氏”的用户,并按照年龄从大到小排序输出

In [4]:
query = """SELECT * FROM user WHERE age BETWEEN 20 AND 30 
AND name LIKE '%Zyan%' ORDER BY age DESC;"""
result = pd.read_sql_query(query, conn)
print(result)



   id     name   sex  age         phone
0   5  Zyan Li  Male   24  987-654-0000
1   6  Zyan Li  Male   22  555-123-6666


5. 写出 SQL 语句,计算 user 表中所有用户的平均年龄

In [5]:
query = 'SELECT AVG(age) AS average_age FROM user'
cursor.execute(query)
result = pd.read_sql_query(query, conn)
print(result)


   average_age
0    23.833333


6. 新建两张表team 表(id,teamName)和score 表(id,teamid,userid,score)。其中score 表中的 teamid 为指向 team表id 的外键，userid 为指向 user表id的外键

In [6]:
cursor.execute('DELETE FROM team')#防止重复添加
cursor.execute('DELETE FROM score')
cursor.execute('DELETE FROM sqlite_sequence WHERE name="team"')
cursor.execute('DELETE FROM sqlite_sequence WHERE name="score"')
conn.commit()

cursor.execute('''
CREATE TABLE IF NOT EXISTS team (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teamName TEXT
)
 
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS score (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    teamid INTEGER,
    userid INTEGER,
    score INTEGER,
    FOREIGN KEY (teamid) REFERENCES team(id),
    FOREIGN KEY (userid) REFERENCES user(id))
''')

conn.commit()

7. 在team表中和score表中插入合适的记录，写出 SQL语句,查询 teamName 为“ECNU”的队伍中，年龄小于 20 的用户们，结果不得为空。

In [7]:


cursor.execute('''INSERT INTO team (teamName) VALUES ('ECNU')''')
cursor.execute('''INSERT INTO team (teamName) VALUES ('SJTU')''')
cursor.executemany('''
INSERT INTO score (teamid, userid, score) VALUES (?, ?, ?)''',
 [(1, 4, 100),(1, 1, 50)])

conn.commit()

query = """
SELECT u.name, u.sex, u.age, u.phone 
FROM user u
JOIN score s ON u.id = s.userid
JOIN team t ON s.teamid = t.id
WHERE t.teamName = 'ECNU' AND u.age < 20;
"""
result = pd.read_sql_query(query, conn)
print(result)

      name   sex  age         phone
0  Zyan Li  Male   19  123-456-7777


8. 写出 SQL 语句,计算 teamName为“ECNU”的总分(假设 score 存在 null值,nul值默认为 0 加入计算)。

In [8]:
query = """
SELECT t.teamName, IFNULL(SUM(s.score), 0) AS total_score
FROM team t
JOIN score s ON t.id = s.teamid
WHERE t.teamName = 'ECNU';
"""
result = pd.read_sql_query(query, conn)
print(result)

  teamName  total_score
0     ECNU          150


9. 写出SQL语句，删除user表中个人信息的记录。

In [9]:
cursor.execute('DELETE FROM user')
conn.commit()